In [1]:
from osgeo import gdal
import numpy as np
import os
import glob

# Function definitions

In [2]:
def CalculateAOI(WestLimitOut, EastLimitOut, NorthLimitOut, SouthLimitOut, AggregationFactor=1,
                 OverallWestLimit=-180, OverallNorthLimit=90):
    # calculates the pixel coords of a given AOI in degrees within the 1km global true-coords
    # rasters (or at least, they must have origin at -180.0, 90.0)
    # returns tuple of tuples ((xmin,xmax),(ymin,ymax))
    
    resolution = 0.008333333333333 * AggregationFactor# 12 threes
    assert EastLimitOut > WestLimitOut
    assert NorthLimitOut > SouthLimitOut
    
    x0 = int((WestLimitOut - OverallWestLimit) / resolution)
    x1 = int(((EastLimitOut - OverallWestLimit) / resolution)+0.5)
    y0 = int((OverallNorthLimit - NorthLimitOut) / resolution)
    y1 = int(((OverallNorthLimit - SouthLimitOut) / resolution) + 0.5)
    
    return ((x0,x1),(y0,y1))

In [1]:
x = (1,2)

In [2]:
isinstance(x,tuple)

True

In [3]:
def CalculateGT(gdalDatasetIn, xLims, yLims, roundToMasterGrid=False,
                OverallWestLimit=-180, OverallNorthLimit=90):
    
    # gdalDatasetIn should be in "true" resolution i.e. 1km = 0.008333333333333 deg
    inputGT = gdalDatasetIn.GetGeoTransform() # assumed in "true" resolution
    topLeftLongIn = inputGT[0]
    topLeftLatIn = inputGT[3]
    resX = inputGT[1]
    resY = inputGT[5]
    
    #print resX, resY
    if not roundToMasterGrid:
        # just calculate the coordinates relative to the source image with its native resolution
        topLeftLongOut = topLeftLongIn + xLims[0] * resX
        topLeftLatOut = topLeftLatIn + yLims[0] * resY # resY will be a negative value
        clippedGT = (topLeftLongOut, resX, 0.0, topLeftLatOut, 0.0, resY)
    
    else:
        # round the coordinates of the input (which must be "true") to those of the imprecisely-defined MG coords
        # i.e. what would the same pixel coords be in real world coords when the pixels have the slightly-wrong 
        # resolution of 0.00833333 set
        #res1K = 0.008333333333333
        #res5k
        masterGridResolution = round(resX, 8)
        assert masterGridResolution != resX
        
        #if resX == res1K:
        #    masterGridResolution = round(resX, 8)
        #elif resX == res1K * 5
        #assert resX == 0.008333333333333
        #assert resY == -0.008333333333333
   
        #masterGridResolution = 0.00833333 # 8 dp
        
        # where does the top left corner of this (Extracted) image sit in terms of number of pixels 
        # from the global top left (or whatever the top left of the source image was)
        topLeftPixelX = (topLeftLongIn - -180) / resX
        # where would the western edge of the input be, when snapped to the mastergrid
        topLeftLongInMG = -180.0 + (topLeftPixelX * masterGridResolution)
        
        topLeftPixelY = (90.0 - topLeftLatIn) / (-resY)
        # where would the northern edge of the input be, when snapped to the mastergrid
        topLeftLatInMG = 89.99994 - (topLeftPixelY * masterGridResolution)
        
        topLeftLongOut = topLeftLongInMG + xLims[0] * masterGridResolution
        topLeftLatOut = topLeftLatInMG - yLims[0] * masterGridResolution
        clippedGT = (topLeftLongOut, masterGridResolution, 0.0, topLeftLatOut, 0.0, -masterGridResolution)
    return clippedGT

In [4]:
def ExtractAOI(gdalDatasetIn, xLims, yLims, roundToMasterGrid=False):
    # returns a tuple containing the array and the geotransform of the image it should be saved to
    
    inputBnd = gdalDatasetIn.GetRasterBand(1)
    inputArr = inputBnd.ReadAsArray(xLims[0], yLims[0], xLims[1] - xLims[0], yLims[1] - yLims[0])
    
    clippedGT = CalculateGT(gdalDatasetIn, xLims, yLims, roundToMasterGrid)
    return (inputArr, clippedGT)

In [11]:
def ExtractAndAlign_Single(infile, outputDir, 
                           west, east, north, south,
                           alignToMasterGrid=False, outFNTag="Clipped",
                           createCompressed=True, maintainExtent = False, 
                           aggregationFactor = 1.0,
                           outNDV = None):
    '''
    Extracts a sub-image from a global 1k image with correct geotransform
    
    The input bbox should be specified in degrees. The output image can optionally 
    have its resolution reset to the incorrect (but consistent with other datasets)
    0.00833333 degrees (as opposed to 0.008333333333333). The output image can have 
    the original global extent (with the clipped-out area set to nodata) or can have
    a new geotransform.
    '''
    
    outputPixelsGlobRef = CalculateAOI(west, east, north, south, aggregationFactor)
    outDrv = gdal.GetDriverByName('GTiff')
    inDS = gdal.Open(infile)
    inGT = inDS.GetGeoTransform()
    inBand = inDS.GetRasterBand(1)
    inDT = inBand.DataType
    inProj = inDS.GetProjection()
    inNDV = inBand.GetNoDataValue()
    inWidth = inDS.RasterXSize
    inHeight = inDS.RasterYSize

    inputPixelsGlobRef = CalculateAOI(inGT[0], inGT[0] + inGT[1] * inDS.RasterXSize,
                                      inGT[3], inGT[3] + inGT[5] * inDS.RasterYSize,
                                      aggregationFactor)
    outL_InputRef = outputPixelsGlobRef[0][0] - inputPixelsGlobRef[0][0]
    outR_InputRef = outputPixelsGlobRef[0][1] - inputPixelsGlobRef[0][0]
    outT_InputRef = outputPixelsGlobRef[1][0] - inputPixelsGlobRef[1][0]
    outB_InputRef = outputPixelsGlobRef[1][1] - inputPixelsGlobRef[1][0]
    assert outL_InputRef >= 0
    assert outR_InputRef > outL_InputRef
    assert outT_InputRef >= 0
    assert outB_InputRef > outT_InputRef

    xLimsToExtract = (outL_InputRef, outR_InputRef)
    yLimsToExtract = (outT_InputRef, outB_InputRef)
    
    res = ExtractAOI(inDS, xLimsToExtract, yLimsToExtract, alignToMasterGrid)

    data = res[0]
    outGT = res[1]

    outFNBase = '.'.join(os.path.basename(infile).split('.')[:-1])
        
    outFNBaseTemplate = "{0}.{1}.tif"
    outFNTemplate = os.path.join(outputDir, outFNBaseTemplate)

    if createCompressed:
        #creationOpts = ["COMPRESS=LZW","TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES"]
        creationOpts = ["COMPRESS=DEFLATE", "ZLEVEL=9", "TILED=YES","SPARSE_OK=TRUE","BIGTIFF=YES"]
    else:
        creationOpts = []
    
    if maintainExtent:
        outRaster = outDrv.Create(outFNTemplate.format(outFNBase,outFNTag),
                                  inWidth, inHeight, 1, inDT,
                                  creationOpts)
        outRaster.SetGeoTransform(inGT)
        outRaster.SetProjection(inProj)
        outBand = outRaster.GetRasterBand(1)
        if inNDV is not None:
            if outNDV is None:
                outBand.SetNoDataValue(inNDV)
            else:
                data[data==inNDV] = outNDV
                outBand.SetNoDataValue(outNDV)
        print (data.shape)
        print outL_InputRef,
        print outT_InputRef
        outBand.WriteArray(data, xoff=outL_InputRef, yoff=outT_InputRef)
        
    else:
        outRaster = outDrv.Create(outFNTemplate.format(outFNBase,outFNTag),
                                  data.shape[1], data.shape[0], 1, inDT,
                                  creationOpts)
        outRaster.SetGeoTransform(outGT)
        outRaster.SetProjection(inProj)
        outBand = outRaster.GetRasterBand(1)
        if inNDV is not None:
            if outNDV is None:
                outBand.SetNoDataValue(inNDV)
            else:
                data[data==inNDV] = outNDV
                outBand.SetNoDataValue(outNDV)
        outBand.WriteArray(data)
    
    outBand.FlushCache()
    outRaster.FlushCache()
    del outBand
    del inBand
    outRaster = None


def ExtractAndAlign(fileList, outputDir, west, east, north, south, 
                    alignToMasterGrid = False, outFNTag="Clip", createCompressed=True):
   for f in fileList:
        ExtractAndAlign_Single(f, outputDir, west, east, north, south,
                               alignToMasterGrid, outFNTag, createCompressed)

# Usage

### Specify input and output directories

###### Input contains the images we are extracting from. Ideally global but at least must have extent covering the required output!

In [6]:
#extractFromDir = r'G:\GapfillingOutputs\Moritz\LST_Day'
#extractToDir = r'G:\GapfillingOutputs\Moritz\LST_Day\Clip_MG'
#extractFromDir = r'I:\MCD43B4_Gapfilled_Output\TCW\Output_30k_2030pc'
#extractToDir = r'J:\Gapfilled_Modis_China\TCW'
extractFromDir = r'E:\Temp\pop\02_processing\06_IHME_Corrected_Grids\with_missing_admin\5km_sum'
extractFromDir = r'E:\Temp\pop\02_processing\02_ihme_admin'
extractToDir = r'C:\Temp\ihme_extract'

###### # create file list

In [7]:
inPattern = (os.path.join(extractFromDir,'*.tif'))
inPattern = (os.path.join(extractFromDir,'IHME_admin_IDs.MG_Matched.tif'))
inFiles = glob.glob(inPattern)

In [8]:
inFiles

['E:\\Temp\\pop\\02_processing\\02_ihme_admin\\IHME_admin_IDs.MG_Matched.tif']

#### # Extract a defined bbox for each of a list of files

##### Run extract and align specifying the output image coordinates and whether we want to align to MGs

In [12]:
#moritzChina = CalculateAOI(73, 136, 54, 18)
#-17.99999999999930000000	37.54166666666520000000	52.04166666666460000000	-34.99999999999860000000

ExtractAndAlign(inFiles, extractToDir, 
                west=-17.99999999999930000000, east=52.04166666666460000000, north=37.54166666666520000000, south=-34.99999999999860000000, 
                alignToMasterGrid=True, outFNTag="Africa", 
                createCompressed=True )
#ExtractAndAlign(inFilesTmp, extractToDir, 
#                west=73, east=136, north=54, south=18, 
#                alignToMasterGrid=True, outFNTag="China_MG", 
#                createCompressed=True )

#### # Extract for ABRAID, recoding to different NDV

In [ ]:
extractFromDir = r'G:\SynopticData\5km\ABRAID_Region\from'
extractToDir = r'G:\SynopticData\5km\ABRAID_Region\to'
# compared to the 1km data what is the resolution we are working with?
aggregationFactor = 5
for infile in inFiles:
    #mth = os.path.basename(infile).split('.')[2]
    north = 85
    south = -60
    ExtractAndAlign_Single(infile, extractToDir, 
                           west=-180, east=180, north=north, south=south,
                           alignToMasterGrid=False, outFNTag="ABRAID_Extent",
                           createCompressed=True, maintainExtent = False,
                           aggregationFactor = aggregationFactor, outNDV = -9999)

In [14]:
infile = r'\\129.67.26.176\map_data\dengue_project\clean_mastergrids\evidence_consensus\Dengue_EBC_June2015\EBC_June2015_MG.tif'
north = 85
south = -60
ExtractAndAlign_Single(infile, r'\\129.67.26.176\map_data\dengue_project\clean_mastergrids\evidence_consensus\Dengue_EBC_June2015',
                       west=-180, east=180, north=north, south=south,
                       alignToMasterGrid=False, outFNTag="Extent",
                       createCompressed=True, maintainExtent=False,
                       aggregationFactor=5, outNDV=-9999)

In [18]:
infile = r'G:\supporting\CoastGlobal.tiff'
north = 85
south = -60
ExtractAndAlign_Single(infile, r'G:\Supporting',
                       west=-180, east=180, north=north, south=south,
                       alignToMasterGrid=False, outFNTag="MGExtent",
                       createCompressed=True, maintainExtent=False,
                       aggregationFactor=1, outNDV=-9999)

#### # Extract MCD43B4 monthlies to seasonally-varying lat limits

In [6]:
# the N-S limits we want by month. Derived empirically.
EVI_NS_Lims = {
    "01":(60,-60),
    "02":(68,-60),
    "03":(80,-60),
    "04":(80,-60),
    "05":(80,-60),
    "06":(80,-60),
    "07":(80,-60),
    "08":(80,-60),
    "09":(80,-60),
    "10":(68,-60),
    "11":(62,-60),
    "12":(60,-60)
}

In [7]:
#extractFromDir = r'E:\MCD43B4\MCD43B4_Gapfilled_Output\EVI\Output_Monthly_Means\5km'
#extractToDir = r'E:\MCD43B4\MCD43B4_Gapfilled_Output\EVI\Output_Monthly_Means\5km_Cutoff'
#extractFromDir = r'I:\MCD43B4_Gapfilled_Output\TCW\Output_Monthly_Means\5km'
#extractToDir = r'I:\MCD43B4_Gapfilled_Output\TCW\Output_Monthly_Means\5km_Cutoff'
extractFromDir = r'G:\Extra\Output\Aggregated\5km'
extractToDir = r'G:\Extra\Output\Aggregated\5km\Clipped'

In [12]:
#inPattern = (os.path.join(extractFromDir,'*.tif'))
inPattern = (os.path.join(extractFromDir,'TC*.tif'))
inFiles = glob.glob(inPattern)

In [13]:
inFiles

['G:\\Extra\\Output\\Aggregated\\5km\\TCB.2015.06.5km.FilledProportion.tif',
 'G:\\Extra\\Output\\Aggregated\\5km\\TCB.2015.06.5km.Mean.tif',
 'G:\\Extra\\Output\\Aggregated\\5km\\TCB.2015.07.5km.FilledProportion.tif',
 'G:\\Extra\\Output\\Aggregated\\5km\\TCB.2015.07.5km.Mean.tif',
 'G:\\Extra\\Output\\Aggregated\\5km\\TCB.2015.08.5km.FilledProportion.tif',
 'G:\\Extra\\Output\\Aggregated\\5km\\TCB.2015.08.5km.Mean.tif',
 'G:\\Extra\\Output\\Aggregated\\5km\\TCB.2015.09.5km.FilledProportion.tif',
 'G:\\Extra\\Output\\Aggregated\\5km\\TCB.2015.09.5km.Mean.tif',
 'G:\\Extra\\Output\\Aggregated\\5km\\TCB.2015.10.5km.FilledProportion.tif',
 'G:\\Extra\\Output\\Aggregated\\5km\\TCB.2015.10.5km.Mean.tif',
 'G:\\Extra\\Output\\Aggregated\\5km\\TCB.2015.11.5km.FilledProportion.tif',
 'G:\\Extra\\Output\\Aggregated\\5km\\TCB.2015.11.5km.Mean.tif',
 'G:\\Extra\\Output\\Aggregated\\5km\\TCB.2015.12.5km.FilledProportion.tif',
 'G:\\Extra\\Output\\Aggregated\\5km\\TCB.2015.12.5km.Mean.tif',
 'G:\\

In [14]:
# compared to the 1km data what is the resolution we are working with?
aggregationFactor = 5
for infile in inFiles:
    mth = os.path.basename(infile).split('.')[2]
    latLims = EVI_NS_Lims[mth]
    north = latLims[0]
    south = latLims[1]
    ExtractAndAlign_Single(infile, extractToDir, 
                           west=-180, east=180, north=north, south=south,
                           alignToMasterGrid=False, outFNTag="Clipped",
                           createCompressed=True, maintainExtent = True,
                           aggregationFactor = aggregationFactor)

(3360L, 8640L)
0 240
(3360L, 8640L)
0 240
(3360L, 8640L)
0 240
(3360L, 8640L)
0 240
(3360L, 8640L)
0 240
(3360L, 8640L)
0 240
(3360L, 8640L)
0 240
(3360L, 8640L)
0 240
(3072L, 8640L)
0 528
(3072L, 8640L)
0 528
(2928L, 8640L)
0 672
(2928L, 8640L)
0 672
(2880L, 8640L)
0 720
(2880L, 8640L)
0 720
(3360L, 8640L)
0 240
(3360L, 8640L)
0 240
(3360L, 8640L)
0 240
(3360L, 8640L)
0 240
(3360L, 8640L)
0 240
(3360L, 8640L)
0 240
(3360L, 8640L)
0 240
(3360L, 8640L)
0 240
(3072L, 8640L)
0 528
(3072L, 8640L)
0 528
(2928L, 8640L)
0 672
(2928L, 8640L)
0 672
(2880L, 8640L)
0 720
(2880L, 8640L)
0 720


#### Mosaic multiple sub rasters into a single output

In [ ]:
inDir = r'C:\Users\zool1301\Documents\Other_Data\Population\WorldPop\Births\raw\2010'
inFiles = glob.glob(os.path.join(inDir,'*.tif'))

In [ ]:
nLimit = -np.inf
sLimit = np.inf
wLimit = np.inf
eLimit = -np.inf

xRes = None
yRes = None
goodFiles = []
for f in inFiles:
    ds = gdal.Open(f)
    gt = ds.GetGeoTransform()
    
    left = gt[0]
    top = gt[3]
    resX = gt[1]
    resY = gt[5]
    width = ds.RasterXSize
    height = ds.RasterYSize
    right = left + resX * width
    bottom = top + resY * height
    
    if xRes is not None:
        if abs(xRes - resX) > 0.000001:
            print "warning res wrong, skipping!"
            print (xRes, resX)
            print f
            continue
    else:
        xRes = resX
    if yRes is not None:
        if abs(yRes - resY) > 0.000001:
            print "warning res wrong, skipping!"
            print (yRes, resY)
            print f
            continue
    else:
        yRes = resY
    goodFiles.append(f)
    if top > nLimit:
        nLimit = top
    if bottom < sLimit:
        sLimit = bottom
    if left < wLimit:
        wLimit = left
    if right > eLimit:
        eLimit = right

In [ ]:
masterGridResolution100m = 0.0008333

In [ ]:
outFN = r'C:\Users\zool1301\Documents\Other_Data\Population\WorldPop\Births\mosaicedimage\Births2010_GDAL.tif'
outDrv = gdal.GetDriverByName('GTiff')
outRaster = outDrv.Create(outFNTemplate.format(outFNBase,outFNTag),
                                  inWidth, inHeight, 1, inDT,
                                  creationOpts)
outRaster.SetGeoTransform(inGT)
outRaster.SetProjection(inProj)
outBand = outRaster.GetRasterBand(1)
if inNDV is not None:
    outBand.SetNoDataValue(inNDV)
print (data.shape)
print outL_InputRef,
print outT_InputRef
outBand.WriteArray(data, xoff=outL_InputRef, yoff=outT_InputRef)